In [1]:
import os
import glob
import pandas as pd
import numpy as np
import re
from collections import Counter
import jieba
jieba.load_userdict('./mydictionary.txt')
#jieba.load_userdict(r'I:\我的雲端硬碟\TCFD\mydictionary.txt')
import ast

from sklearn.preprocessing import LabelEncoder
from gensim.models import Word2Vec
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC

from sklearn.metrics import classification_report
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_val_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score

Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\Owner\AppData\Local\Temp\jieba.cache
Loading model cost 0.655 seconds.
Prefix dict has been built successfully.


# 新增欄位(third)association_word, '(third)top_association_word'

In [1]:
top_association_words_bank = {
    '0g': [
        ('氣候', '風險'), ('風險', '管理'), ('管理', '風險'), ('氣候', '管理'), ('風險', '氣候'), 
        ('管理', '氣候'), ('氣候', '相關'), ('相關', '風險'), ('本行', '風險'), ('相關', '管理')
    ],
    
    '1s': [
        ('氣候', '風險'), ('實體', '風險'), ('風險', '氣候'), ('轉型', '風險'), ('情境', '風險'), 
        ('分析', '風險'), ('情境', '分析'), ('氣候變遷', '風險'), ('相關', '風險'), ('風險', '影響')
    ],
    
    '2r': [
        ('氣候', '風險'), ('管理', '風險'), ('風險', '管理'), ('風險', '氣候'), ('相關', '風險'), 
        ('本行', '風險'), ('氣候', '管理'), ('評估', '風險'), ('氣候變遷', '風險'), ('實體', '風險')
         
    ],
    
    '3m': [
        ('氣候', '風險'), ('風險', '氣候'), ('實體', '風險'), ('指標', '目標'), ('高碳排', '產業'), 
        ('氣候', '目標'), ('相關', '風險'), ('本行', '氣候'), ('溫室氣體', '排放'), ('氣候', '相關')
    ]
}

In [ ]:
top_association_words_life = {
    '0g': [
        ('氣候', '風險'), ('風險', '管理'), ('管理', '風險'), ('氣候', '管理'), ('相關', '風險'), 
        ('氣候', '相關'), ('風險', '氣候'), ('管理', '氣候'), ('公司', '風險'), ('相關', '管理')
    ],
    
    '1s': [
        ('氣候', '風險'), ('相關', '風險'), ('風險', '氣候'), ('氣候變遷', '風險'), ('實體', '風險'), 
        ('氣候', '相關'), ('情境', '風險'), ('影響', '風險'), ('轉型', '風險'), ('分析', '風險')
    ],
    
    '2r': [
        ('氣候', '風險'), ('管理', '風險'), ('相關', '風險'), ('風險', '管理'), ('風險', '氣候'), 
        ('公司', '風險'), ('氣候', '管理'), ('氣候變遷', '風險'), ('氣候', '相關'), ('風險', '相關')
         
    ],
    
    '3m': [
        ('氣候', '風險'), ('氣候變遷', '風險'), ('相關', '風險'), ('風險', '氣候'), ('公司', '風險'), 
        ('風險', '管理'), ('管理', '風險'), ('氣候', '管理'), ('實體', '風險'), ('氣候', '相關')
    ]
}

In [2]:
top_association_words_property = {
    '0g': [
        ('氣候', '風險'), ('氣候', '相關'), ('風險', '管理'), ('相關', '風險'), ('氣候', '管理'),  
        ('管理', '風險'), ('風險', '氣候'), ('相關', '管理'), ('管理', '氣候'), ('風險', '相關')
    ],
    
    '1s': [
        ('氣候', '風險'), ('相關', '風險'), ('風險', '氣候'), ('氣候', '相關'), ('評估', '風險'), 
        ('管理', '風險'), ('風險', '管理'), ('相關', '氣候'), ('影響', '風險'), ('公司', '風險')
    ],
    
    '2r': [
        ('氣候', '風險'), ('管理', '風險'), ('風險', '管理'), ('相關', '風險'), ('公司', '風險'), 
        ('風險', '氣候'), ('氣候', '管理'), ('氣候', '相關'), ('保險', '風險'), ('氣候變遷', '風險')
         
    ],
    
    '3m': [
        ('氣候', '風險'), ('相關', '風險'), ('風險', '管理'), ('氣候變遷', '風險'), ('氣候', '相關'), 
        ('公司', '風險'), ('風險', '氣候'), ('管理', '風險'), ('氣候', '管理'), ('風險', '相關')
    ]
}

In [3]:
tuples_allcat = []
for key, tuples in top_association_words_property.items():
    print(key, tuples)
    for tuple in tuples:
        tuples_allcat.append(tuple)
print(len(tuples_allcat), len(set(tuples_allcat)))

0g [('氣候', '風險'), ('氣候', '相關'), ('風險', '管理'), ('相關', '風險'), ('氣候', '管理'), ('管理', '風險'), ('風險', '氣候'), ('相關', '管理'), ('管理', '氣候'), ('風險', '相關')]
1s [('氣候', '風險'), ('相關', '風險'), ('風險', '氣候'), ('氣候', '相關'), ('評估', '風險'), ('管理', '風險'), ('風險', '管理'), ('相關', '氣候'), ('影響', '風險'), ('公司', '風險')]
2r [('氣候', '風險'), ('管理', '風險'), ('風險', '管理'), ('相關', '風險'), ('公司', '風險'), ('風險', '氣候'), ('氣候', '管理'), ('氣候', '相關'), ('保險', '風險'), ('氣候變遷', '風險')]
3m [('氣候', '風險'), ('相關', '風險'), ('風險', '管理'), ('氣候變遷', '風險'), ('氣候', '相關'), ('公司', '風險'), ('風險', '氣候'), ('管理', '風險'), ('氣候', '管理'), ('風險', '相關')]
40 16


In [ ]:
#top_third_association_words = {
#    '0g': [],
#    '1s': [('氣候', '風險', '影響'), ('氣候', '風險', '機會')],
#    '2r': [],
#    '3m': [('本行', '氣候', '風險'), ('產業', '氣候', '風險'), ('氣候', '風險', '產業'), ('氣候', '指標', '目標')]
#}

In [5]:
rootFolderList = [
    #'./data/bankContent/0g',
    #'./data/bankContent/1s',
    #'./data/bankContent/2r',
    #'./data/bankContent/3m',
    #'./data/insuranceContent/l/0g',
    #'./data/insuranceContent/l/1s',
    #'./data/insuranceContent/l/2r',
    #'./data/insuranceContent/l/3m',
    './data/insuranceContent/p/0g',
    './data/insuranceContent/p/1s',
    './data/insuranceContent/p/2r',
    './data/insuranceContent/p/3m'
        ]

mapping = {
    '0g': '治理',
    '1s': '策略',
    '2r': '風險',
    '3m': '指標'
}

# 讀取中文停用詞表
stopwords = set()
with open('./baiducnhit_stopwords.txt', 'r', encoding='utf8') as stopword_file:
    for line in stopword_file:
        stopwords.add(line.strip())

unionwords=[]
top_category_words_dict = {}

dataAndClass = {
    'Filename': [], 'Text': [], 'Words': [], 'Category': [], 
    'association_word': [], 'top_association_word': [],
    #'third_association_word': [], 'top_third_association_word': []
}

for rootFolder in rootFolderList:
    category = os.path.basename(rootFolder)
    chineseCategory = mapping.get(category, category)

    for root, dirs, files in os.walk(rootFolder):
        # print(root, dirs, files) # ./data/bankContent/0g [] ['2022_b_Bank of Panhsin_tcfd_g.txt']
        txtcount = 0
        txtcountcontent = 0

        category_words = []
        for txt in files:
            if txt.endswith('.txt'):
                txtcount += 1
                with open(os.path.join(root, txt), encoding="utf8") as file:
                    txtdata = file.read()
                    if len(txtdata) > 0:
                        txtcountcontent += 1
                        txtdatanew = re.sub(r'[\n\t\s.◆•]', '', txtdata)
                        txtdatanew = re.sub(r'[^\u4e00-\u9fa5]', '', txtdatanew)

                        words = list(jieba.cut(txtdatanew, cut_all=False))
                        words = [w for w in words if len(w) >= 2 and w not in stopwords]
                        category_words.extend(words)
                        unionwords.extend(words)
                        dataAndClass['Text'].append(txtdatanew)
                        dataAndClass['Category'].append(chineseCategory)
                        dataAndClass['Words'].append(words)
                        Filenamewotxt = txt.replace('.txt', '')
                        dataAndClass['Filename'].append(Filenamewotxt)
                        # 找出每個樣本中的二連詞
                        ass_words = [(words[i], words[i+1]) for i in range(len(words)-1)]
                        dataAndClass['association_word'].append(ass_words)
                        # 篩選符合該類十大二連詞
                        top_ass_word = [ass_word for ass_word in ass_words if ass_word in top_association_words_property.get(category, [])]
                        dataAndClass['top_association_word'].append(top_ass_word)

                        ## 找出每個樣本中的三連詞
                        #third_ass_words = [(words[i], words[i+1], words[i+2]) for i in range(len(words)-2)]
                        #dataAndClass['third_association_word'].append(third_ass_words)

                        ## 篩選符合該類大三連詞
                        #top_third_ass_word = [third_ass_word for third_ass_word in third_ass_words if third_ass_word in top_third_association_words.get(category, [])]
                        #dataAndClass['top_third_association_word'].append(top_third_ass_word)
        print(f"{chineseCategory} txtcount: {txtcount}, txtcountcontent: {txtcountcontent}")

## 特徵篩選
# 挑選每類詞頻前10的詞彙
        top_category_words = [word for word, freq in Counter(category_words).most_common(10)]
        print(f"{chineseCategory} top words: {top_category_words}")
        top_category_words_dict[chineseCategory] = top_category_words

治理 txtcount: 11, txtcountcontent: 11
治理 top words: ['風險', '氣候', '管理', '相關', '永續', '氣候變遷', '公司', '治理', '機會', '委員會']
策略 txtcount: 11, txtcountcontent: 11
策略 top words: ['風險', '氣候', '情境', '相關', '管理', '評估', '公司', '影響', '轉型', '投資']
風險 txtcount: 11, txtcountcontent: 11
風險 top words: ['風險', '管理', '氣候', '相關', '公司', '保險', '投資', '氣候變遷', '營運', '影響']
指標 txtcount: 11, txtcountcontent: 11
指標 top words: ['目標', '排放', '公司', '溫室氣體', '氣候', '指標', '管理', '風險', '投資', '組合']


In [6]:
dataAndClassdf = pd.DataFrame(dataAndClass)
dataAndClassdf.columns

Index(['Filename', 'Text', 'Words', 'Category', 'association_word',
       'top_association_word'],
      dtype='object')

In [7]:
dataAndClassdf

,Filename,Text,Words,Category,association_word,top_association_word
0,2022_p_fubon_tcfd_g,治理氣候治理架構本公司設有永續經營推動小組依部門性質將公司部門分組成立六大工作小組公司治理及...,"[治理, 氣候, 治理, 架構, 公司, 設有, 永續, 經營, 推動, 小組, 部門, 性...",治理,"[(治理, 氣候), (氣候, 治理), (治理, 架構), (架構, 公司), (公司, ...","[(風險, 相關), (氣候, 風險), (風險, 管理), (風險, 管理), (風險, ..."
1,2022_p_CTBC Holding_tcfd_g,四治理氣候治理架構績效考核與薪酬機制董事中信金控董事會績效評估辦法業已明訂績效評估之衡量項目...,"[治理, 氣候, 治理, 架構, 績效, 考核, 薪酬, 機制, 董事, 中信, 金控, 董...",治理,"[(治理, 氣候), (氣候, 治理), (治理, 架構), (架構, 績效), (績效, ...","[(管理, 風險), (氣候, 風險), (氣候, 相關), (相關, 風險), (氣候, ..."
2,2022_p_chubb_tcfd_g,安達產險臺灣分公司作為在安達集團監督下營運的當地法人實體設有地方風險管理委員會即委員會負責實...,"[安達產險, 分公司, 安達, 集團, 監督, 營運, 法人, 實體, 設有, 地方, 風險...",治理,"[(安達產險, 分公司), (分公司, 安達), (安達, 集團), (集團, 監督), (...","[(風險, 管理), (氣候, 風險), (風險, 管理), (風險, 管理), (風險, ..."
3,2022_p_taian_tcfd_g,氣候治理架構本公司氣候變遷風險管理辦法於年底經董事會審議通過另本報告書將於每年依法規規定時程...,"[氣候, 治理, 架構, 公司, 氣候變遷, 風險, 管理, 辦法, 年底, 董事會, 審議...",治理,"[(氣候, 治理), (治理, 架構), (架構, 公司), (公司, 氣候變遷), (氣候...","[(風險, 管理), (氣候, 相關), (相關, 風險), (風險, 管理), (氣候, ..."
4,2022_p_chungkuo_tcfd_g,氣候變遷治理兆豐金控已於年月簽署支持氣候相關財務揭露倡議依循架構揭露氣候相關資訊兆豐保險為兆...,"[氣候變遷, 治理, 兆豐金控, 簽署, 支持, 氣候, 相關, 財務, 揭露, 倡議, 依...",治理,"[(氣候變遷, 治理), (治理, 兆豐金控), (兆豐金控, 簽署), (簽署, 支持),...","[(氣候, 相關), (氣候, 相關), (氣候, 相關), (風險, 管理), (氣候, ..."
5,2022_p_AIG_tcfd_g,治理氣候變遷與公司治理本公司為之台灣分公司本公司為外國公司之台灣分公司而無設置董事會因此由高...,"[治理, 氣候變遷, 公司, 治理, 公司, 台灣, 分公司, 公司, 外國, 公司, 之台...",治理,"[(治理, 氣候變遷), (氣候變遷, 公司), (公司, 治理), (治理, 公司), (...","[(風險, 管理), (氣候, 相關), (相關, 風險), (氣候, 相關), (相關, ..."
6,2022_p_tfmic_tcfd_g,二治理理事會對氣候相關風險與機會監督為使合作社永續發展並考慮本社規模理事會為氣候環境風險管理...,"[治理, 理事, 氣候, 相關, 風險, 機會, 監督, 合作社, 永續, 發展, 並考慮,...",治理,"[(治理, 理事), (理事, 氣候), (氣候, 相關), (相關, 風險), (風險, ...","[(氣候, 相關), (相關, 風險), (風險, 管理), (氣候, 風險), (風險, ..."
7,2022_p_south-china_tcfd_g,氣候治理隨著極端天氣與氣候變遷日漸嚴峻各國政府及企業應更積極思考如何共同抑制全球升溫議題年各...,"[氣候, 治理, 極端, 天氣, 氣候變遷, 日漸, 嚴峻, 各國, 政府, 企業, 思考,...",治理,"[(氣候, 治理), (治理, 極端), (極端, 天氣), (天氣, 氣候變遷), (氣候...","[(管理, 氣候), (氣候, 風險), (氣候, 相關), (風險, 管理), (氣候, ..."
8,2022_p_cathay_esg_g,氣候治理國泰產險氣候變遷治理架構對於氣候相關風險與機會之治理國泰產險以低碳永續經營為目標由董...,"[氣候, 治理, 國泰產險, 氣候變遷, 治理, 架構, 氣候, 相關, 風險, 機會, 治...",治理,"[(氣候, 治理), (治理, 國泰產險), (國泰產險, 氣候變遷), (氣候變遷, 治理...","[(氣候, 相關), (相關, 風險), (氣候, 相關), (氣候, 相關), (相關, ..."
9,2022_p_BNP_tcfd_g,氣候治理與監督氣候治理組織架構與權責本分公司氣候風險管理組織架構包括經理人總經理風險管理委員...,"[氣候, 治理, 監督, 氣候, 治理, 組織, 架構, 權責本, 分公司, 氣候, 風險,...",治理,"[(氣候, 治理), (治理, 監督), (監督, 氣候), (氣候, 治理), (治理, ...","[(氣候, 風險), (風險, 管理), (風險, 管理), (氣候, 風險), (風險, ..."


In [7]:
dataAndClassdf[['Category', 'Text', 'Words', 'association_word', 'top_association_word', 'third_association_word', 'top_third_association_word']][118:123]

,Category,Text,Words,association_word,top_association_word,third_association_word,top_third_association_word
118,指標,低碳營運臺銀積極因應氣候變遷為最小化環境衝擊對內精進營運效率提升營運據點實體風險減緩與調適措...,"[低碳, 營運, 銀積極, 因應, 氣候變遷, 最小化, 環境, 衝擊, 對內, 精進, 營...","[(低碳, 營運), (營運, 銀積極), (銀積極, 因應), (因應, 氣候變遷), (...","[(實體, 風險), (氣候, 風險), (溫室氣體, 排放), (溫室氣體, 排放), (...","[(低碳, 營運, 銀積極), (營運, 銀積極, 因應), (銀積極, 因應, 氣候變遷)...",[]
119,指標,指標與目標關鍵指標及目標綠色營運本行長期推動節能減碳措施目前規劃預計於年碳排量下降以年為基準...,"[指標, 目標, 關鍵, 指標, 目標, 綠色, 營運, 本行, 推動, 節能, 減碳, 措...","[(指標, 目標), (目標, 關鍵), (關鍵, 指標), (指標, 目標), (目標, ...","[(指標, 目標), (指標, 目標)]","[(指標, 目標, 關鍵), (目標, 關鍵, 指標), (關鍵, 指標, 目標), (指標...",[]
120,指標,氣候治理與風險管理本行董事會為氣候變遷相關議題的最高治理單位另外也透過風險管理委員會與永續發...,"[氣候, 治理, 風險, 管理, 本行, 董事會, 氣候變遷, 相關, 議題, 治理, 單位...","[(氣候, 治理), (治理, 風險), (風險, 管理), (管理, 本行), (本行, ...","[(氣候, 風險), (相關, 風險), (氣候, 相關), (氣候, 相關), (相關, ...","[(氣候, 治理, 風險), (治理, 風險, 管理), (風險, 管理, 本行), (管理...","[(本行, 氣候, 風險), (本行, 氣候, 風險), (本行, 氣候, 風險), (本行..."
121,指標,投融資風險管理金融業之氣候相關風險主要來自投融資業務且潛在衝擊不容小覷為針對高氣候風險產業進...,"[投融資, 風險, 管理, 金融, 業之, 氣候, 相關, 風險, 投融資, 業務且, 衝擊...","[(投融資, 風險), (風險, 管理), (管理, 金融), (金融, 業之), (業之,...","[(氣候, 相關), (相關, 風險), (氣候, 風險), (高碳排, 產業), (高碳排...","[(投融資, 風險, 管理), (風險, 管理, 金融), (管理, 金融, 業之), (金...","[(氣候, 風險, 產業), (氣候, 指標, 目標)]"
122,指標,自身營運面溫室氣體排放量與再生能源本行每年進行溫室氣體盤查並通過溫室氣體盤查查證透過全面性的...,"[營運, 溫室氣體, 排放量, 再生能源, 本行, 溫室氣體, 盤查, 並通過, 溫室氣體,...","[(營運, 溫室氣體), (溫室氣體, 排放量), (排放量, 再生能源), (再生能源, ...","[(指標, 目標), (氣候, 相關), (指標, 目標), (氣候, 風險), (指標, ...","[(營運, 溫室氣體, 排放量), (溫室氣體, 排放量, 再生能源), (排放量, 再生能...",[]


# 新增欄位字詞指標

In [8]:
df = pd.read_excel('./basic_stats/file_stats_property.xlsx')
df

,Filename,Company,Category,Total_Count_Unigram,Different_Unigrams,TTR_Unigram500,Entropy_Unigram500,Simpson_Unigram500,Total_Count_Bigram,Different_Bigrams,TTR_Bigram500,Entropy_Bigram500,Simpson_Bigram500
0,2022_p_hotai_tcfd_g,hotai,g,845,252,165,4.687791,0.013765,315,156,133,4.417904,0.019940
1,2022_p_south-china_tcfd_g,south-china,g,1183,350,198,4.898448,0.011580,427,225,169,4.595251,0.020760
2,2022_p_fubon_tcfd_g,fubon,g,851,264,177,4.835582,0.011304,309,177,151,4.642028,0.014370
3,2022_p_CTBC Holding_tcfd_g,CTBC Holding,g,919,288,190,4.973140,0.008891,286,170,149,4.694533,0.012706
4,2022_p_BNP_tcfd_g,BNP,g,954,274,171,4.679402,0.014663,356,159,131,4.258773,0.027298
5,2022_p_AIG_tcfd_g,AIG,g,652,192,142,4.509755,0.016367,221,90,84,3.834752,0.038827
6,2022_p_tfmic_tcfd_g,tfmic,g,180,89,86,4.099132,0.022168,70,40,40,3.385213,0.045696
7,2022_p_chubb_tcfd_g,chubb,g,484,177,143,4.553923,0.015598,175,96,92,4.116691,0.026085
8,2022_p_taian_tcfd_g,taian,g,448,192,151,4.668352,0.012580,165,106,102,4.284959,0.018590
9,2022_p_cathay_esg_g,cathay,g,638,227,164,4.721248,0.012969,229,133,122,4.406230,0.018317


In [9]:
selected_columns = [
    'Filename', 'Total_Count_Unigram', 'Different_Unigrams', 'TTR_Unigram500',
    'Entropy_Unigram500', 'Simpson_Unigram500', 'Total_Count_Bigram', 'Different_Bigrams',
    'TTR_Bigram500', 'Entropy_Bigram500', 'Simpson_Bigram500'
]
df_selected = df[selected_columns]
df_selected

,Filename,Total_Count_Unigram,Different_Unigrams,TTR_Unigram500,Entropy_Unigram500,Simpson_Unigram500,Total_Count_Bigram,Different_Bigrams,TTR_Bigram500,Entropy_Bigram500,Simpson_Bigram500
0,2022_p_hotai_tcfd_g,845,252,165,4.687791,0.013765,315,156,133,4.417904,0.019940
1,2022_p_south-china_tcfd_g,1183,350,198,4.898448,0.011580,427,225,169,4.595251,0.020760
2,2022_p_fubon_tcfd_g,851,264,177,4.835582,0.011304,309,177,151,4.642028,0.014370
3,2022_p_CTBC Holding_tcfd_g,919,288,190,4.973140,0.008891,286,170,149,4.694533,0.012706
4,2022_p_BNP_tcfd_g,954,274,171,4.679402,0.014663,356,159,131,4.258773,0.027298
5,2022_p_AIG_tcfd_g,652,192,142,4.509755,0.016367,221,90,84,3.834752,0.038827
6,2022_p_tfmic_tcfd_g,180,89,86,4.099132,0.022168,70,40,40,3.385213,0.045696
7,2022_p_chubb_tcfd_g,484,177,143,4.553923,0.015598,175,96,92,4.116691,0.026085
8,2022_p_taian_tcfd_g,448,192,151,4.668352,0.012580,165,106,102,4.284959,0.018590
9,2022_p_cathay_esg_g,638,227,164,4.721248,0.012969,229,133,122,4.406230,0.018317


In [10]:
dataAndClassdf = pd.merge(dataAndClassdf, df_selected, on='Filename', how='left')
dataAndClassdf.columns
dataAndClassdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44 entries, 0 to 43
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Filename              44 non-null     object 
 1   Text                  44 non-null     object 
 2   Words                 44 non-null     object 
 3   Category              44 non-null     object 
 4   association_word      44 non-null     object 
 5   top_association_word  44 non-null     object 
 6   Total_Count_Unigram   44 non-null     int64  
 7   Different_Unigrams    44 non-null     int64  
 8   TTR_Unigram500        44 non-null     int64  
 9   Entropy_Unigram500    44 non-null     float64
 10  Simpson_Unigram500    44 non-null     float64
 11  Total_Count_Bigram    44 non-null     int64  
 12  Different_Bigrams     44 non-null     int64  
 13  TTR_Bigram500         44 non-null     int64  
 14  Entropy_Bigram500     44 non-null     float64
 15  Simpson_Bigram500     44 

# 新增欄位Words_top10_in_cat

In [11]:
words_top10_list = []
#篩每類前十大詞
for i, doc in enumerate(dataAndClass['Words']):
    #print(i, words)
    category = dataAndClass['Category'][i]
    top_words = top_category_words_dict.get(category, [])
    words_top10 = [word for word in doc if word in top_words]
    #print(words_top10)
    words_top10_list.append(words_top10)
dataAndClassdf['Words_top10'] = words_top10_list
#dataAndClassdf = pd.DataFrame(dataAndClass)
#dataAndClassdf.to_excel('dataAndClassdf_top10.xlsx', index=False)
dataAndClassdf.head(1)


,Filename,Text,Words,Category,association_word,top_association_word,Total_Count_Unigram,Different_Unigrams,TTR_Unigram500,Entropy_Unigram500,Simpson_Unigram500,Total_Count_Bigram,Different_Bigrams,TTR_Bigram500,Entropy_Bigram500,Simpson_Bigram500,Words_top10
0,2022_p_fubon_tcfd_g,治理氣候治理架構本公司設有永續經營推動小組依部門性質將公司部門分組成立六大工作小組公司治理及...,"[治理, 氣候, 治理, 架構, 公司, 設有, 永續, 經營, 推動, 小組, 部門, 性...",治理,"[(治理, 氣候), (氣候, 治理), (治理, 架構), (架構, 公司), (公司, ...","[(風險, 相關), (氣候, 風險), (風險, 管理), (風險, 管理), (風險, ...",851,264,177,4.835582,0.011304,309,177,151,4.642028,0.01437,"[治理, 氣候, 治理, 公司, 永續, 公司, 公司, 治理, 永續, 永續, 氣候變遷,..."


# 新增欄位Words_top100

In [12]:
#篩前五百大詞
# 計算整個語料庫的詞頻
all_words = [word for doc in dataAndClass['Words'] for word in doc]
word_freq = Counter(all_words)
print(f'文本未篩選時，{sum(Counter(all_words).values())}個詞，有{len(Counter(all_words))}種詞')
# 挑選詞頻前 100 的詞彙
top_words = [word for word, freq in word_freq.most_common(100)]
print(len(top_words))
dataAndClassdf['Words_top100'] = [[word for word in doc if word in top_words] for doc in dataAndClass['Words']]
#dataAndClassdf = pd.DataFrame(dataAndClass)
#dataAndClassdf.to_excel('dataAndClassdf_top100.xlsx', index=False)
dataAndClassdf.head(1)

文本未篩選時，20703個詞，有3725種詞
100


,Filename,Text,Words,Category,association_word,top_association_word,Total_Count_Unigram,Different_Unigrams,TTR_Unigram500,Entropy_Unigram500,Simpson_Unigram500,Total_Count_Bigram,Different_Bigrams,TTR_Bigram500,Entropy_Bigram500,Simpson_Bigram500,Words_top10,Words_top100
0,2022_p_fubon_tcfd_g,治理氣候治理架構本公司設有永續經營推動小組依部門性質將公司部門分組成立六大工作小組公司治理及...,"[治理, 氣候, 治理, 架構, 公司, 設有, 永續, 經營, 推動, 小組, 部門, 性...",治理,"[(治理, 氣候), (氣候, 治理), (治理, 架構), (架構, 公司), (公司, ...","[(風險, 相關), (氣候, 風險), (風險, 管理), (風險, 管理), (風險, ...",851,264,177,4.835582,0.011304,309,177,151,4.642028,0.01437,"[治理, 氣候, 治理, 公司, 永續, 公司, 公司, 治理, 永續, 永續, 氣候變遷,...","[治理, 氣候, 治理, 架構, 公司, 永續, 推動, 公司, 公司, 治理, 金融, 環..."


# 新增欄位Words_top500

In [13]:
#篩前五百大詞
# 計算整個語料庫的詞頻
all_words = [word for doc in dataAndClass['Words'] for word in doc]
word_freq = Counter(all_words)
print(f'文本未篩選時，{sum(Counter(all_words).values())}個詞，有{len(Counter(all_words))}種詞')
# 挑選詞頻前 500 的詞彙
top_words = [word for word, freq in word_freq.most_common(500)]
print(len(top_words))
dataAndClassdf['Words_top500'] = [[word for word in doc if word in top_words] for doc in dataAndClass['Words']]
#dataAndClassdf = pd.DataFrame(dataAndClass)
#dataAndClassdf.to_excel('dataAndClassdf_top10_top500.xlsx', index=False)
dataAndClassdf.head(1)

文本未篩選時，20703個詞，有3725種詞
500


,Filename,Text,Words,Category,association_word,top_association_word,Total_Count_Unigram,Different_Unigrams,TTR_Unigram500,Entropy_Unigram500,Simpson_Unigram500,Total_Count_Bigram,Different_Bigrams,TTR_Bigram500,Entropy_Bigram500,Simpson_Bigram500,Words_top10,Words_top100,Words_top500
0,2022_p_fubon_tcfd_g,治理氣候治理架構本公司設有永續經營推動小組依部門性質將公司部門分組成立六大工作小組公司治理及...,"[治理, 氣候, 治理, 架構, 公司, 設有, 永續, 經營, 推動, 小組, 部門, 性...",治理,"[(治理, 氣候), (氣候, 治理), (治理, 架構), (架構, 公司), (公司, ...","[(風險, 相關), (氣候, 風險), (風險, 管理), (風險, 管理), (風險, ...",851,264,177,4.835582,0.011304,309,177,151,4.642028,0.01437,"[治理, 氣候, 治理, 公司, 永續, 公司, 公司, 治理, 永續, 永續, 氣候變遷,...","[治理, 氣候, 治理, 架構, 公司, 永續, 推動, 公司, 公司, 治理, 金融, 環...","[治理, 氣候, 治理, 架構, 公司, 永續, 經營, 推動, 小組, 部門, 公司, 部..."


In [14]:
dataAndClassdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44 entries, 0 to 43
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Filename              44 non-null     object 
 1   Text                  44 non-null     object 
 2   Words                 44 non-null     object 
 3   Category              44 non-null     object 
 4   association_word      44 non-null     object 
 5   top_association_word  44 non-null     object 
 6   Total_Count_Unigram   44 non-null     int64  
 7   Different_Unigrams    44 non-null     int64  
 8   TTR_Unigram500        44 non-null     int64  
 9   Entropy_Unigram500    44 non-null     float64
 10  Simpson_Unigram500    44 non-null     float64
 11  Total_Count_Bigram    44 non-null     int64  
 12  Different_Bigrams     44 non-null     int64  
 13  TTR_Bigram500         44 non-null     int64  
 14  Entropy_Bigram500     44 non-null     float64
 15  Simpson_Bigram500     44 

# X_Word vectors
Word2Vec文字轉詞向量

## 拆解後的詞列表訓練Word2Vec模型

In [15]:
import numpy as np
from gensim.models import Word2Vec

# 將tuple拆解為詞的列表
def flatten_association_list(association_list):
    """
    將包含tuple的二連詞列表拆解為詞的列表，以便用於Word2Vec的訓練。
    """
    flattened_list = []
    for doc in association_list:
        flattened_doc = []
        for bigram in doc:
            if isinstance(bigram, type(tuple)):
                # 將tuple中的詞拆開，加入到列表中
                flattened_doc.extend(bigram)  
        flattened_list.append(flattened_doc)
    return flattened_list

In [30]:
dataAndClassdf[['Text','Words','Words_top10','Words_top100','Words_top500','top_association_word']].head(2)#[117:118]

,Text,Words,Words_top10,Words_top100,Words_top500,top_association_word
0,治理因應全球暖化現象加劇安聯集團致力於應對氣候變遷相關之風險議題並將減緩全球暖化訴求及推展應...,"[治理, 因應, 全球, 暖化, 現象, 加劇安聯, 集團, 致力, 氣候變遷, 相關, 風...","[治理, 氣候變遷, 相關, 風險, 氣候, 相關, 風險, 氣候, 風險, 董事會, 公司...","[治理, 因應, 全球, 集團, 氣候變遷, 相關, 風險, 議題, 全球, 氣候, 相關,...","[治理, 因應, 全球, 暖化, 集團, 致力, 氣候變遷, 相關, 風險, 議題, 並將,...","[(相關, 風險), (氣候, 相關), (相關, 風險), (氣候, 風險), (氣候, ..."
1,氣候治理隨著極端天氣與氣候變遷問題日漸嚴峻各國政府及企業開始思考採取更積極的行動來抑制全球均...,"[氣候, 治理, 極端, 天氣, 氣候變遷, 日漸, 嚴峻, 各國, 政府, 企業, 思考,...","[氣候, 治理, 氣候變遷, 氣候, 氣候, 治理, 氣候變遷, 氣候, 相關, 氣候變遷,...","[氣候, 治理, 氣候變遷, 企業, 全球, 氣候, 達成, 全球, 金融, 全球, 發展,...","[氣候, 治理, 極端, 天氣, 氣候變遷, 各國, 政府, 企業, 全球, 各國, 聯合國...","[(氣候, 相關), (風險, 管理), (風險, 管理), (風險, 管理), (氣候, ..."


In [98]:
print(f'===raw column values===')
wordDftoList_top10 = dataAndClassdf['Words_top10'].tolist()
print(len(wordDftoList_top10),wordDftoList_top10)
#wordDftoList_top100= dataAndClassdf['Words_top100'].tolist()
#print(len(wordDftoList_top100))#,wordDftoList_top100)
#wordDftoList_top500 = dataAndClassdf['Words_top500'].tolist()
#print(len(wordDftoList_top500))
#wordDftoList_association = dataAndClassdf['top_association_word'].tolist()
#print(len(wordDftoList_association),wordDftoList_association)
#wordDftoList_third_association = dataAndClassdf['top_third_association_word'].tolist()
#print(len(wordDftoList_third_association),wordDftoList_third_association)

# 將二連詞的tuple拆開，變成單詞列表
print(f'===flatten association list===')
#flattened_association_list = flatten_association_list(wordDftoList_association)
#print(len(flattened_association_list),flattened_association_list)
#flattened_third_association_list = flatten_association_list(wordDftoList_third_association)
#print(len(flattened_third_association_list),flattened_third_association_list)

===raw column values===
44 [['治理', '氣候', '治理', '公司', '永續', '公司', '公司', '治理', '永續', '永續', '氣候變遷', '風險', '相關', '氣候', '風險', '管理', '氣候變遷', '相關', '永續', '永續', '管理', '永續', '管理', '永續', '永續', '永續', '相關', '永續', '公司', '永續', '風險', '管理', '風險', '氣候變遷', '風險', '機會', '相關', '風險', '管理', '委員會', '永續', '氣候變遷', '風險', '氣候變遷', '風險', '氣候', '風險', '相關', '氣候', '風險', '氣候', '風險', '相關', '氣候變遷', '風險', '治理', '氣候', '風險', '機會', '氣候', '風險', '永續', '相關', '氣候變遷', '風險', '公司', '風險'], ['治理', '氣候', '治理', '公司', '永續', '氣候', '永續', '氣候', '相關', '永續', '氣候', '永續', '氣候', '公司', '治理', '永續', '永續', '永續', '永續', '管理', '管理', '風險', '永續', '氣候變遷', '永續', '氣候', '風險', '機會', '治理', '永續', '永續', '氣候', '相關', '風險', '氣候', '風險'], ['風險', '管理', '委員會', '氣候', '風險', '管理', '風險', '管理', '委員會', '風險', '管理', '委員會', '氣候', '相關', '風險', '管理', '風險', '管理', '委員會', '風險', '管理', '委員會', '風險', '氣候', '氣候', '相關', '風險', '機會', '相關', '風險', '管理', '委員會', '公司', '治理', '氣候', '相關', '風險', '管理', '風險', '風險', '機會', '風險', '管理', '委員會', '氣候', '相關', '風險', '管理', '氣候', '風險'], ['氣候', '治理', '公司', '氣候變遷

In [ ]:
# Example
#top10_list = [['氣候', '氣候變遷']] #, '氣候', '風險', '管理', '氣候', '風險'
#top100_list = [['治理', '氣候', '治理', '架構', '全球', '氣候變遷']]#, '環境', '產業'
#top500_list =[['治理', '氣候', '治理', '架構', '全球', '氣候變遷']]
#association_list = [['氣候', '風險']]
#third_association_list= [['氣候', '指標', '目標']]

#wordlist_to_learn_check=[
#    top10 +top100 +top500 +association +third_association 
#    for top10, top100, top500, association, third_association  in zip(top10_list, top100_list, top500_list, association_list, third_association_list)]
#print(wordlist_to_learn_check)
#print(len(set(wordlist_to_learn_check[0])))

[['氣候', '氣候變遷', '治理', '氣候', '治理', '架構', '全球', '氣候變遷', '治理', '氣候', '治理', '架構', '全球', '氣候變遷', '氣候', '風險', '氣候', '指標', '目標']]
8


In [ ]:
#allw_check=[]
#for wlist in wordlist_to_learn_check:
#    print(wlist)
#    for word in wlist:
#        allw_check.append(word)
#print(f'total {len(allw_check)} words, {len(set(allw_check))} features learned')

['氣候', '氣候變遷', '治理', '氣候', '治理', '架構', '全球', '氣候變遷', '治理', '氣候', '治理', '架構', '全球', '氣候變遷', '氣候', '風險', '氣候', '指標', '目標']
total 19 words, 8 features learned


In [99]:
#wordlist_to_learn = wordDftoList_top500

wordlist_to_learn = [
    top10_list
    #+top100_list
    #+top500_list 
    #+ association_list 
    #+ third_association_list  # 使用 + 符號合併每個樣本的三個列表
    for
    top10_list, 
    #top100_list,
    #top500_list, 
    #association_list, 
    #third_association_list 
    in zip(
        wordDftoList_top10,
        #wordDftoList_top100,
        #wordDftoList_top500,
        #flattened_association_list, 
        #flattened_third_association_list
        )
]

# 打印合併後的結果
print(len(wordlist_to_learn))#, wordlist_to_learn)

44


In [100]:
allw=[]
for wlist in wordlist_to_learn[-1:]:
    for word in wlist:
        allw.append(word)
print(f'total {len(allw)} words, {len(set(allw))} features learned')

total 92 words, 10 features learned


In [101]:
for top10_list in wordDftoList_top10[-1:]:
    print(f'top10  {len(top10_list)} words{top10_list}')
#for top100_list in wordDftoList_top100[-1:]:
#    print(f'top100 {len(top100_list)} words{top100_list}')
#for top500_list in wordDftoList_top500[-1:]:
#    print(f'top500 {len(top500_list)} words{top500_list}')
#for association_list in wordDftoList_association[-1:]:
#    print(f'association {len(association_list)*2} words{association_list}')
for combine in wordlist_to_learn[-1:]:
    print(f'total {len(combine)} words{combine}')

top10  92 words['目標', '指標', '目標', '溫室氣體', '管理', '管理', '公司', '溫室氣體', '溫室氣體', '溫室氣體', '排放', '溫室氣體', '目標', '目標', '排放', '指標', '氣候', '風險', '指標', '目標', '組合', '排放', '溫室氣體', '排放', '投資', '管理', '組合', '排放', '管理', '排放', '組合', '公司', '公司', '組合', '排放', '排放', '投資', '組合', '排放', '投資', '組合', '投資', '組合', '組合', '排放', '排放', '排放', '組合', '組合', '排放', '組合', '組合', '投資', '排放', '組合', '排放', '組合', '排放', '投資', '組合', '排放', '排放', '排放', '管理', '排放', '管理', '管理', '管理', '投資', '組合', '排放', '組合', '目標', '目標', '組合', '排放', '目標', '目標', '風險', '指標', '風險', '組合', '風險', '風險', '風險', '風險', '風險', '氣候', '風險', '風險', '風險', '投資']
total 92 words['目標', '指標', '目標', '溫室氣體', '管理', '管理', '公司', '溫室氣體', '溫室氣體', '溫室氣體', '排放', '溫室氣體', '目標', '目標', '排放', '指標', '氣候', '風險', '指標', '目標', '組合', '排放', '溫室氣體', '排放', '投資', '管理', '組合', '排放', '管理', '排放', '組合', '公司', '公司', '組合', '排放', '排放', '投資', '組合', '排放', '投資', '組合', '投資', '組合', '組合', '排放', '排放', '排放', '組合', '組合', '排放', '組合', '組合', '投資', '排放', '組合', '排放', '組合', '排放', '投資', '組合', '排放', '排放', '排放', '管理', '排放', '管理

In [102]:
allw=[]
for wlist in wordlist_to_learn:
    for word in wlist:
        allw.append(word)
print(f'total {len(allw)} words, {len(set(allw))} features learned')

total 4991 words, 22 features learned


In [103]:
random_seed = 42
np.random.seed(random_seed)

word2vec_model = Word2Vec(sentences = wordlist_to_learn,#flattened_association_list + wordDftoList_top10, 
                          vector_size=100, 
                          window=20, 
                          min_count=1, 
                          workers=1, # 使用單執行緒
                          seed=random_seed)
print(word2vec_model)
vector_size = word2vec_model.vector_size

# 打印Word2Vec學習到的詞彙
words = word2vec_model.wv.key_to_index
print(words)
print(f'Word2Vec features:{len(words)}')


Word2Vec<vocab=22, vector_size=100, alpha=0.025>
{'風險': 0, '氣候': 1, '管理': 2, '相關': 3, '公司': 4, '投資': 5, '影響': 6, '情境': 7, '氣候變遷': 8, '評估': 9, '保險': 10, '轉型': 11, '目標': 12, '營運': 13, '永續': 14, '排放': 15, '溫室氣體': 16, '指標': 17, '治理': 18, '組合': 19, '機會': 20, '委員會': 21}
Word2Vec features:22


## 計算文檔向量(詞向量平均)

In [70]:
# 計算平均詞向量，對於每個二連詞
def calculate_tuple_avg_vector(tuple_words, model):
    """
    計算二連詞中兩個詞的平均詞向量。
    """
    valid_vectors = []
    
    # 對於二連詞中的每個詞，提取其詞向量
    for word in tuple_words:
        if word in model.wv:
            vector = model.wv[word]
            valid_vectors.append(vector)
            #print(f"Word: {word}, Vector: {vector}")
        else:
            print(f"Word {word} not found in Word2Vec model")
    
    # 計算平均詞向量
    if valid_vectors:
        avg_vector = np.mean(valid_vectors, axis=0)
        print(f"Average vector for tuple {tuple_words}: {avg_vector}")
        return avg_vector
    else:
        # 如果沒有找到有效的詞，返回零向量
        return np.zeros(model.vector_size)


In [88]:
#wordlist_to_convert = wordDftoList_top500

wordlist_to_convert = [
    top10_list
    #+top100_list
    #+top500_list
    + association_list 
    #+ third_association_list # 使用 + 符號合併每個樣本的n個列表
    for
    top10_list,
    #top100_list,
    #top500_list, 
    association_list,
    #third_association_list 
    in zip(
        wordDftoList_top10,
        #wordDftoList_top100,
        #wordDftoList_top500,
        wordDftoList_association,
        #wordDftoList_third_association
        )
]
print(wordlist_to_convert[:1])#

[['治理', '氣候', '治理', '公司', '永續', '公司', '公司', '治理', '永續', '永續', '氣候變遷', '風險', '相關', '氣候', '風險', '管理', '氣候變遷', '相關', '永續', '永續', '管理', '永續', '管理', '永續', '永續', '永續', '相關', '永續', '公司', '永續', '風險', '管理', '風險', '氣候變遷', '風險', '機會', '相關', '風險', '管理', '委員會', '永續', '氣候變遷', '風險', '氣候變遷', '風險', '氣候', '風險', '相關', '氣候', '風險', '氣候', '風險', '相關', '氣候變遷', '風險', '治理', '氣候', '風險', '機會', '氣候', '風險', '永續', '相關', '氣候變遷', '風險', '公司', '風險', ('風險', '相關'), ('氣候', '風險'), ('風險', '管理'), ('風險', '管理'), ('風險', '管理'), ('氣候', '風險'), ('風險', '相關'), ('氣候', '風險'), ('氣候', '風險'), ('風險', '相關'), ('氣候', '風險'), ('氣候', '風險')]]


In [89]:
for top10_list in wordDftoList_top10[-1:]:
    print(len(top10_list),top10_list)
#for top100_list in wordDftoList_top100[-1:]:
#    print(len(top100_list),top100_list)
#for top500_list in wordDftoList_top500[-1:]:
#    print(len(top500_list),top500_list)
for association_list in wordDftoList_association[-1:]:
    print(len(association_list), association_list)
for combine in wordlist_to_convert[-1:]:
    print(len(combine),combine)

92 ['目標', '指標', '目標', '溫室氣體', '管理', '管理', '公司', '溫室氣體', '溫室氣體', '溫室氣體', '排放', '溫室氣體', '目標', '目標', '排放', '指標', '氣候', '風險', '指標', '目標', '組合', '排放', '溫室氣體', '排放', '投資', '管理', '組合', '排放', '管理', '排放', '組合', '公司', '公司', '組合', '排放', '排放', '投資', '組合', '排放', '投資', '組合', '投資', '組合', '組合', '排放', '排放', '排放', '組合', '組合', '排放', '組合', '組合', '投資', '排放', '組合', '排放', '組合', '排放', '投資', '組合', '排放', '排放', '排放', '管理', '排放', '管理', '管理', '管理', '投資', '組合', '排放', '組合', '目標', '目標', '組合', '排放', '目標', '目標', '風險', '指標', '風險', '組合', '風險', '風險', '風險', '風險', '風險', '氣候', '風險', '風險', '風險', '投資']
1 [('氣候', '相關')]
93 ['目標', '指標', '目標', '溫室氣體', '管理', '管理', '公司', '溫室氣體', '溫室氣體', '溫室氣體', '排放', '溫室氣體', '目標', '目標', '排放', '指標', '氣候', '風險', '指標', '目標', '組合', '排放', '溫室氣體', '排放', '投資', '管理', '組合', '排放', '管理', '排放', '組合', '公司', '公司', '組合', '排放', '排放', '投資', '組合', '排放', '投資', '組合', '投資', '組合', '組合', '排放', '排放', '排放', '組合', '組合', '排放', '組合', '組合', '投資', '排放', '組合', '排放', '組合', '排放', '投資', '組合', '排放', '排放', '排放', '管理', '排放', '管理', '管理

In [90]:
# 用來存儲每個文件的平均向量
X_word_vectors = []
for doc in wordlist_to_convert:#wordDftoList_association[37:38]
    print(doc)
    doc_valid_vectors = []
    for ass_words in doc:
        #print(ass_words)        
        if isinstance(ass_words, type(tuple)):# 如果是tuple，計算其平均詞向量
            tuple_avg_vector = calculate_tuple_avg_vector(ass_words, word2vec_model)
            #print(f"Association words: {ass_words}, Average Vector: {tuple_avg_vector}")
            doc_valid_vectors.append(tuple_avg_vector)
        elif ass_words in word2vec_model.wv:
            vector =  word2vec_model.wv[ass_words]
            doc_valid_vectors.append(vector)
        else: #not found in Word2Vec model, assigning zero vector.")
            doc_valid_vectors.append(np.zeros(word2vec_model.vector_size))
    if doc_valid_vectors:
        doc_avg_vector = np.mean(doc_valid_vectors, axis=0)
        print(f"Average vector for doc: {doc_avg_vector}\n")
    else:
        # 如果沒有找到有效的詞，返回零向量
        doc_avg_vector = np.zeros(word2vec_model.vector_size)
        print(f"Average vector for doc: {doc_avg_vector}\n")
    X_word_vectors.append(doc_avg_vector)
#print(f"Average vector for all docs: {X}")

# 將 X 轉換為 NumPy 矩陣，並打印其形狀
X_word_vectors = np.array(X_word_vectors)
print(f"Word vectors shape: {X_word_vectors.shape}")  # 應該是 [123, vector_size]

['治理', '氣候', '治理', '公司', '永續', '公司', '公司', '治理', '永續', '永續', '氣候變遷', '風險', '相關', '氣候', '風險', '管理', '氣候變遷', '相關', '永續', '永續', '管理', '永續', '管理', '永續', '永續', '永續', '相關', '永續', '公司', '永續', '風險', '管理', '風險', '氣候變遷', '風險', '機會', '相關', '風險', '管理', '委員會', '永續', '氣候變遷', '風險', '氣候變遷', '風險', '氣候', '風險', '相關', '氣候', '風險', '氣候', '風險', '相關', '氣候變遷', '風險', '治理', '氣候', '風險', '機會', '氣候', '風險', '永續', '相關', '氣候變遷', '風險', '公司', '風險', ('風險', '相關'), ('氣候', '風險'), ('風險', '管理'), ('風險', '管理'), ('風險', '管理'), ('氣候', '風險'), ('風險', '相關'), ('氣候', '風險'), ('氣候', '風險'), ('風險', '相關'), ('氣候', '風險'), ('氣候', '風險')]
Average vector for tuple ('風險', '相關'): [-0.03469298  0.2670454  -0.36355758  0.24022686  0.10743441 -0.0442344
  0.07956015  0.13144876 -0.5282653   0.02929143 -0.25855845  0.6321305
 -0.00728328 -0.05848122  0.25454295  0.09888625 -0.35597914 -0.07967936
  0.49857882  0.02082742  0.14785087 -0.01030211 -0.04707385  0.6855303
  0.19323567 -0.1718909  -0.43910152  0.18166938  0.0694176  -0.16489655
  0.04360279 

# X_Scaled numeric

In [91]:
X_numeric = dataAndClassdf[[
    'Total_Count_Unigram', 
    'Different_Unigrams', 
    'TTR_Unigram500', 'Entropy_Unigram500', 'Simpson_Unigram500',
    'Total_Count_Bigram', 
    'Different_Bigrams', 
    'TTR_Bigram500', 'Entropy_Bigram500', 'Simpson_Bigram500'                    
]]
print(f"Numeric features shape: {X_numeric.shape}")

Numeric features shape: (44, 10)


In [92]:
from sklearn.preprocessing import MinMaxScaler #特徵縮放到 [0, 1] 的範圍內，避免數值型特徵對模型過度影響。
scaler = MinMaxScaler()
X_numeric_scaled = scaler.fit_transform(X_numeric)

In [258]:
#from sklearn.preprocessing import StandardScaler #特徵標準化，使其均值為 0，標準差為 1。
#scaler = StandardScaler()
#X_numeric_scaled = scaler.fit_transform(X_numeric)

# X

In [93]:
# 合併詞向量與數值型變數
#X = X_word_vectors
#X = X_numeric_scaled
X = np.hstack((X_word_vectors, X_numeric_scaled))
print(f"Combined feature shape: {X.shape}")

Combined feature shape: (44, 110)


# y

In [ ]:
#def combine_sr(category):
#    if category in ["策略", "風險"]:  # 需要合併的類別名稱
#        return "策略風險"
#    else:
#        return category

In [ ]:
#def combine_gs(category):
#    if category in ["治理", "策略"]:  # 需要合併的類別名稱
#        return "治理策略"
#    else:
#        return "風險指標"

In [94]:
y = dataAndClassdf['Category']
#print(y.shape, y.unique()) # (173,)


In [ ]:
## 更新類別資料
#y = y.apply(combine_sr)

In [ ]:
#print("類別分布：")
#print(y.value_counts())

類別分布：
治理    11
策略    11
風險    11
指標    11
Name: Category, dtype: int64


In [95]:
# Encode the string labels into numerical labels
label_encoder = LabelEncoder()
y = np.array(label_encoder.fit_transform(y))
print("類別對應：")
for i, class_name in enumerate(label_encoder.classes_):
    print(f"{i} -> {class_name}")

類別對應：
0 -> 指標
1 -> 治理
2 -> 策略
3 -> 風險


# Modeling

## Do not execute! Save 500 times train test split indices

In [ ]:
#train_test_indices = {
#    'train_indices': [],
#    'test_indices': []
#}

#numTrials = 500
#train_sample = 0
#test_sample = 0
#traindistribution = 0
#testdistribution = 0
#for i in range(numTrials):
#    # 保存訓練和測試集的索引
#    # 使用train_test_split來切分資料，但只記錄索引
#    X_train_indices, X_test_indices, _, _ = train_test_split(np.arange(len(X)), y, test_size=0.2, stratify=y, random_state=None)
#    train_test_indices['train_indices'].append(X_train_indices.tolist())
#    train_test_indices['test_indices'].append(X_test_indices.tolist())
#    # 查看全部資料三種類別比例
#    traindistribution += pd.Series(y[X_train_indices]).value_counts(normalize=True)
#    testdistribution += pd.Series(y[X_test_indices]).value_counts(normalize=True)
#    train_sample += len(X_train_indices)
#    test_sample += len(X_test_indices)

## 保存切分索引
#np.save('train_test_indices_classification_property_simulation500.npy', train_test_indices, allow_pickle=True)

#print(f"Train sample:{train_sample}\nTest sample:{test_sample}")
#print(f"Train distribution:\n{traindistribution/numTrials}\nTest distribution:\n{testdistribution/numTrials}")

Train sample:17500
Test sample:4500
Train distribution:
3    0.249943
1    0.250000
0    0.250057
2    0.250000
Name: proportion, dtype: float64
Test distribution:
2    0.250000
1    0.250000
3    0.250222
0    0.249778
Name: proportion, dtype: float64


In [ ]:
## 檢查數據類型和內容
#print(f"train_test_indices type: {type(train_test_indices)}")
#print('===1/500 sampling===')
#print(f"{len(train_test_indices['train_indices'][0])} Train indices for the first sampling(1/500): {train_test_indices['train_indices'][0]}")
#print(f"{len(train_test_indices['test_indices'][0])} Test indices for the first sampling(1/500): {train_test_indices['test_indices'][0]}")

train_test_indices type: <class 'dict'>
===1/500 sampling===
35 Train indices for the first sampling(1/500): [42, 39, 32, 20, 29, 23, 12, 1, 27, 10, 4, 28, 24, 18, 36, 35, 38, 30, 41, 40, 37, 17, 8, 7, 3, 34, 6, 21, 26, 25, 19, 13, 5, 9, 14]
9 Test indices for the first sampling(1/500): [0, 15, 2, 22, 11, 33, 16, 31, 43]


In [ ]:
## 檢查有哪些鍵存在於 train_test_indices 中
#print("Available keys:", train_test_indices.keys())

Available keys: dict_keys(['train_indices', 'test_indices'])


# Use saved indices to cross validation

In [96]:
traindistribution = 0
testdistribution = 0
train_sample = 0
test_sample = 0

random_seed = 42
precision_list = []
recall_list = []
f1_score_list = []
accuracy_list = []
conf_matrix_sum = np.zeros((len(label_encoder.classes_), len(label_encoder.classes_)))
# 加載之前存儲的資料切分索引
train_test_indices = np.load('./train_test_indices_classification_property_simulation500.npy', allow_pickle=True).item()
print(f"{len(train_test_indices['train_indices'][0])} Train indices for the first sampling(1/500): {train_test_indices['train_indices'][0]}")
print(f"{len(train_test_indices['test_indices'][0])} Test indices for the first sampling(1/500): {train_test_indices['test_indices'][0]}")
# 使用相同的資料切分進行模擬
for i, (X_train_indices, X_test_indices) in enumerate(zip(
    train_test_indices['train_indices'], 
    train_test_indices['test_indices']
)):
    X_train = X[X_train_indices]
    X_test = X[X_test_indices]
    y_train = y[X_train_indices]
    y_test = y[X_test_indices]

    # 查看全部資料四種類別比例
    traindistribution += pd.Series(y[X_train_indices]).value_counts(normalize=True)
    testdistribution += pd.Series(y[X_test_indices]).value_counts(normalize=True)
    train_sample += len(X_train_indices)
    test_sample += len(X_test_indices)
    
    #lr = LogisticRegression(penalty="l2",
    #                        class_weight='balanced',
    #                        max_iter=1000)
    #lr.fit(X_train, y_train)
    #y_pred = lr.predict(X_test)

#    #rf = RandomForestClassifier(n_estimators=200, random_state=1080)
#    #rf.fit(X_train, y_train)
#    #y_pred = rf.predict(X_test)

#    #nb = MultinomialNB()
#    #nb.fit(X_train, y_train)
#    #y_pred = nb.predict(X_test)

    svm = SVC(random_state=random_seed)
    svm.fit(X_train, y_train)
    y_pred = svm.predict(X_test)

#    #xgb = XGBClassifier()
#    #xgb.fit(X_train, y_train)
#    #y_pred = xgb.predict(X_test)

    # 計算綜合指標
    precision, recall, f1_score, support = precision_recall_fscore_support(y_test, y_pred, average='weighted', zero_division=1)  # Set zero_division to 1
    accuracy = accuracy_score(y_test, y_pred)
    precision_list.append(precision)
    recall_list.append(recall)
    f1_score_list.append(f1_score)
    accuracy_list.append(accuracy)
    conf_matrix = metrics.confusion_matrix(y_test, y_pred, labels=np.arange(len(label_encoder.classes_)))
    conf_matrix_sum += conf_matrix
print(f"Train sample:{train_sample}\nTest sample:{test_sample}")
print(f"Train distribution:\n{traindistribution/100}\nTest distribution:\n{testdistribution/100}")

# Class names for labels
class_names = ['指標', '治理', '策略', '風險']
#class_names = ['指標', '治理', '策略風險'] #0 -> 指標, 1 -> 治理, 2 -> 策略風險
#class_names = ['治理策略', '風險指標']#0 -> 治理策略, 1 -> 風險指標
conf_matrix_df = pd.DataFrame(conf_matrix_sum.astype(int), index=class_names, columns=class_names)
print(conf_matrix_df)

# 計算標準差
std_precision = np.std(precision_list)
std_recall = np.std(recall_list)
std_f1_score = np.std(f1_score_list)
std_accuracy = np.std(accuracy_list)

# 計算平均綜合指標
average_precision = np.mean(precision_list)
average_recall = np.mean(recall_list)
average_f1_score = np.mean(f1_score_list)
average_accuracy = np.mean(accuracy_list)

print(f"Average Precision:{average_precision:.3f}, Average Recall:{average_recall:.3f}, Average F1 Score:{average_f1_score:.3f}, Average Accuracy:{average_accuracy:.3f}")
print(f"Standard Deviation Precision:{std_precision:.3f}, Standard Deviation Recall:{std_recall:.3f}, Standard Deviation F1 Score:{std_f1_score:.3f}, Standard Deviation Accuracy:{std_accuracy:.3f}")

35 Train indices for the first sampling(1/500): [29, 19, 32, 5, 38, 8, 0, 23, 35, 9, 4, 12, 20, 22, 36, 27, 33, 13, 39, 6, 34, 1, 18, 42, 17, 26, 28, 21, 25, 3, 30, 14, 41, 2, 37]
9 Test indices for the first sampling(1/500): [16, 10, 24, 40, 15, 7, 11, 31, 43]
Train sample:17500
Test sample:4500
Train distribution:
3    1.249714
1    1.250000
0    1.250286
2    1.250000
Name: proportion, dtype: float64
Test distribution:
2    1.250000
1    1.250000
3    1.251111
0    1.248889
Name: proportion, dtype: float64
     指標   治理   策略   風險
指標  918  145   47   14
治理    0  821   70  234
策略    0  522  325  278
風險    0  513  278  335
Average Precision:0.730, Average Recall:0.533, Average F1 Score:0.537, Average Accuracy:0.533
Standard Deviation Precision:0.119, Standard Deviation Recall:0.127, Standard Deviation F1 Score:0.148, Standard Deviation Accuracy:0.127


In [97]:
print(conf_matrix_df)

     指標   治理   策略   風險
指標  918  145   47   14
治理    0  821   70  234
策略    0  522  325  278
風險    0  513  278  335


# Ensemble (Word Vector + Numeric)

In [133]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.ensemble import VotingClassifier

# 初始化詞向量模型和數值特徵模型
word_vector_model = SVC(probability=True, random_state=random_seed)
numeric_model = SVC(probability=True, random_state=random_seed)

In [134]:
precision_list_v = []
recall_list_v = []
f1_score_list_v = []
accuracy_list_v = []
conf_matrix_sum_v = np.zeros((len(label_encoder.classes_), len(label_encoder.classes_)))

In [135]:
train_test_indices = np.load(r'J:\我的雲端硬碟\TCFD\train_test_indices.npy', allow_pickle=True).item()
print(f"{len(train_test_indices['train_indices'][0])} Train indices for the first sampling(1/100): {train_test_indices['train_indices'][0]}")
print(f"{len(train_test_indices['test_indices'][0])} Test indices for the first sampling(1/100): {train_test_indices['test_indices'][0]}")
# 使用相同的資料切分進行模擬
for i, (X_train_indices, X_test_indices) in enumerate(zip(
    train_test_indices['train_indices'], 
    train_test_indices['test_indices']
)):
    # 拆分數據為詞向量和數值特徵
    X_train_word_vectors = X_word_vectors[X_train_indices]
    X_test_word_vectors = X_word_vectors[X_test_indices]

    X_train_numeric = X_numeric_scaled[X_train_indices]
    X_test_numeric = X_numeric_scaled[X_test_indices]

    y_train = y[X_train_indices]
    y_test = y[X_test_indices]

    # 訓練兩個模型
    word_vector_model.fit(X_train_word_vectors, y_train)
    numeric_model.fit(X_train_numeric, y_train)

    # 建立兩個模型的 Voting Classifier
    voting_clf = VotingClassifier(estimators=[
        ('word_vector_model', word_vector_model),
        ('numeric_model', numeric_model)
    ], voting='soft')  # 'soft' 使用概率融合，'hard' 使用票數融合

    # 訓練模型
    voting_clf.fit(np.hstack((X_train_word_vectors, X_train_numeric)), y_train)

    # 預測
    y_pred_v= voting_clf.predict(np.hstack((X_test_word_vectors, X_test_numeric)))

    # 計算綜合指標
    precision_v, recall_v, f1_score_v, support_v = precision_recall_fscore_support(y_test, y_pred_v, average='weighted', zero_division=1)  # Set zero_division to 1
    accuracy_v = accuracy_score(y_test, y_pred_v)
    precision_list_v.append(precision_v)
    recall_list_v.append(recall_v)
    f1_score_list_v.append(f1_score_v)
    accuracy_list_v.append(accuracy_v)
    conf_matrix_v = metrics.confusion_matrix(y_test, y_pred_v, labels=np.arange(len(label_encoder.classes_)))
    conf_matrix_sum_v += conf_matrix_v

98 Train indices for the first sampling(1/100): [1, 30, 72, 33, 121, 2, 111, 23, 26, 86, 109, 80, 42, 35, 73, 32, 34, 0, 77, 18, 51, 52, 90, 87, 5, 94, 28, 71, 57, 14, 82, 64, 68, 113, 117, 89, 41, 93, 112, 96, 110, 84, 122, 20, 7, 119, 54, 48, 31, 108, 60, 95, 46, 61, 56, 39, 44, 24, 36, 97, 55, 79, 21, 29, 58, 38, 59, 22, 16, 91, 67, 88, 120, 37, 8, 78, 9, 98, 116, 103, 69, 65, 74, 75, 63, 17, 106, 15, 70, 115, 118, 101, 11, 99, 3, 4, 40, 92]
25 Test indices for the first sampling(1/100): [76, 19, 104, 10, 114, 81, 6, 107, 27, 45, 53, 50, 83, 12, 49, 13, 100, 43, 25, 85, 66, 62, 47, 105, 102]


In [137]:
print(f"Train sample:{train_sample}\nTest sample:{test_sample}")
print(f"Train distribution:\n{traindistribution/100}\nTest distribution:\n{testdistribution/100}")

# Class names for labels
class_names = ['指標', '治理', '策略', '風險']
conf_matrix_df_v = pd.DataFrame(conf_matrix_sum_v.astype(int), index=class_names, columns=class_names)
print(conf_matrix_df_v)

# 計算標準差
std_precision_v = np.std(precision_list_v)
std_recall_v = np.std(recall_list_v)
std_f1_score_v = np.std(f1_score_list_v)
std_accuracy_v = np.std(accuracy_list_v)

# 計算平均綜合指標
average_precision_v = np.mean(precision_list_v)
average_recall_v = np.mean(recall_list_v)
average_f1_score_v = np.mean(f1_score_list_v)
average_accuracy_v = np.mean(accuracy_list_v)

print(f"Average Precision:{average_precision_v:.3f}, Average Recall:{average_recall_v:.3f}, Average F1 Score:{average_f1_score_v:.3f}, Average Accuracy:{average_accuracy_v:.3f}")
print(f"Standard Deviation Precision:{std_precision_v:.3f}, Standard Deviation Recall:{std_recall_v:.3f}, Standard Deviation F1 Score:{std_f1_score_v:.3f}, Standard Deviation Accuracy:{std_accuracy_v:.3f}")

Train sample:9800
Test sample:100
Train distribution:
3    0.251327
2    0.252041
1    0.251735
0    0.244898
dtype: float64
Test distribution:
1    0.2532
3    0.2548
0    0.2400
2    0.2520
dtype: float64
     指標   治理   策略   風險
指標  491    0   93   16
治理    0  615    0   18
策略   98    0  456   76
風險    0   40  149  448
Average Precision:0.822, Average Recall:0.804, Average F1 Score:0.800, Average Accuracy:0.804
Standard Deviation Precision:0.073, Standard Deviation Recall:0.072, Standard Deviation F1 Score:0.075, Standard Deviation Accuracy:0.072


In [138]:
print(conf_matrix_df_v)

     指標   治理   策略   風險
指標  491    0   93   16
治理    0  615    0   18
策略   98    0  456   76
風險    0   40  149  448
